# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,767.113356,-0.813817,1234,766.299539,767.927173,1532.599078,1535.854346
6,160.948830,0.609273,1234,161.558104,160.339557,323.116208,320.679114
7,898.385998,-0.941149,1234,897.444849,899.327148,1794.889698,1798.654295
8,149.132649,-0.146978,1234,148.985671,149.279627,297.971343,298.559253
9,989.655582,0.988559,1234,990.644141,988.667023,1981.288282,1977.334046
10,767.262542,-0.188428,1234,767.074114,767.450970,1534.148228,1534.901941
11,98.637692,-0.120631,1234,98.517061,98.758323,197.034122,197.516647
12,712.528045,0.376917,1234,712.904962,712.151129,1425.809925,1424.302257
13,993.102605,-0.739923,1234,992.362682,993.842528,1984.725364,1987.685056
14,661.381125,-0.750609,1234,660.630516,662.131734,1321.261033,1324.263468


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-766.299539,767.927173
6,-161.558104,160.339557
7,-897.444849,899.327148
8,-148.985671,149.279627
9,-990.644141,988.667023
10,-767.074114,767.450970
11,-98.517061,98.758323
12,-712.904962,712.151129
13,-992.362682,993.842528
14,-660.630516,662.131734


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-766.299539,767.927173
6,-161.558104,160.339557
7,-897.444849,899.327148
8,-148.985671,149.279627
9,-990.644141,988.667023
10,-767.074114,767.450970
11,-98.517061,98.758323
12,-712.904962,712.151129
13,-992.362682,993.842528
14,-660.630516,662.131734


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-766.29953905,  767.92717299],
       [-161.55810375,  160.33955718],
       [-897.44484923,  899.32714751],
       [-148.98567132,  149.27962655],
       [-990.64414083,  988.66702295],
       [-767.07411414,  767.4509703 ],
       [ -98.5170609 ,   98.75832328],
       [-712.90496238,  712.15112857],
       [-992.36268196,  993.84252824],
       [-660.6305165 ,  662.13173406]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1530.599078,-1535.854346
6,321.116208,-320.679114
7,1792.889698,-1798.654295
8,295.971343,-298.559253
9,1979.288282,-1977.334046
10,1532.148228,-1534.901941
11,195.034122,-197.516647
12,1423.809925,-1424.302257
13,1982.725364,-1987.685056
14,1319.261033,-1324.263468


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
